In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('/home/guozijun/kaggle/Taxi Duration/data/train.csv')
test_data = pd.read_csv('/home/guozijun/kaggle/Taxi Duration/data/test.csv')

In [3]:
train_data['store_and_fwd_flag'] = train_data['store_and_fwd_flag'].map(lambda x: 0 if x=='N' else 1)
test_data['store_and_fwd_flag'] = test_data['store_and_fwd_flag'].map(lambda x: 0 if x=='N' else 1)

In [4]:
#时间特征
from datetime import datetime
train_data['pickup_datetime'] = train_data['pickup_datetime'].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
train_data['pickup_month'] = train_data['pickup_datetime'].map(lambda x: x.month)
train_data['pickup_weekday'] = train_data['pickup_datetime'].map(lambda x: x.weekday())
train_data['pickup_hour'] = train_data['pickup_datetime'].map(lambda x: x.hour)

test_data['pickup_datetime'] = test_data['pickup_datetime'].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
test_data['pickup_month'] = test_data['pickup_datetime'].map(lambda x: x.month)
test_data['pickup_weekday'] = test_data['pickup_datetime'].map(lambda x: x.weekday())
test_data['pickup_hour'] = test_data['pickup_datetime'].map(lambda x: x.hour)

In [5]:
#对聚合之后的特征排序
def sort_groupby(df):
    s = df[[len(df.columns)-1]].columns.values[0]
    df = df.sort_values(by=s,ascending=False)
    df[s+'_sort'] = range(1,len(df)+1)
    return df


In [6]:
#时间特征
# pickup_month_number
# pickup_weekday_number
# pickup_hour_number
# pickup_weekhour_number
train_data['pickup_number'] = 1 # 后续删除
pickup_month_number = train_data.groupby(['pickup_month'],as_index=False)['pickup_number'].sum()
pickup_month_number.rename(columns={'pickup_number':'pickup_month_number'},inplace=True)
pickup_weekday_number = train_data.groupby(['pickup_weekday'],as_index=False)['pickup_number'].sum()
pickup_weekday_number.rename(columns={'pickup_number':'pickup_weekday_number'},inplace=True)
pickup_hour_number = train_data.groupby(['pickup_hour'],as_index=False)['pickup_number'].sum()
pickup_hour_number.rename(columns={'pickup_number':'pickup_hour_number'},inplace=True)
pickup_weekhour_number = train_data.groupby(['pickup_weekday','pickup_hour'],as_index=False)['pickup_number'].sum()
pickup_weekhour_number.rename(columns={'pickup_number':'pickup_weekhour_number'},inplace=True)

pickup_month_number = sort_groupby(pickup_month_number)
pickup_weekday_number = sort_groupby(pickup_weekday_number)
pickup_hour_number = sort_groupby(pickup_hour_number)
pickup_weekhour_number = sort_groupby(pickup_weekhour_number)

train_data = pd.merge(train_data, pickup_month_number, how='left', on = ['pickup_month'])
train_data = pd.merge(train_data, pickup_weekday_number, how='left', on = ['pickup_weekday'])
train_data = pd.merge(train_data, pickup_hour_number, how='left', on = ['pickup_hour'])
train_data = pd.merge(train_data, pickup_weekhour_number, how='left', on = ['pickup_weekday','pickup_hour'])

test_data = pd.merge(test_data, pickup_month_number, how='left', on = ['pickup_month'])
test_data = pd.merge(test_data, pickup_weekday_number, how='left', on = ['pickup_weekday'])
test_data = pd.merge(test_data, pickup_hour_number, how='left', on = ['pickup_hour'])
test_data = pd.merge(test_data, pickup_weekhour_number, how='left', on = ['pickup_weekday','pickup_hour'])


In [7]:
#时间特征
# 月平均出行时长，日平均出行时长，周平均出行时长
trip_duration_m = train_data.groupby(['pickup_month'],as_index=False)['trip_duration'].mean()
trip_duration_m.rename(columns={'trip_duration':'trip_duration_m'},inplace=True)
trip_duration_w = train_data.groupby(['pickup_weekday'],as_index=False)['trip_duration'].mean()
trip_duration_w.rename(columns={'trip_duration':'trip_duration_w'},inplace=True)
trip_duration_h = train_data.groupby(['pickup_hour'],as_index=False)['trip_duration'].mean()
trip_duration_h.rename(columns={'trip_duration':'trip_duration_h'},inplace=True)
trip_duration_wh = train_data.groupby(['pickup_weekday','pickup_hour'],as_index=False)['trip_duration'].mean()
trip_duration_wh.rename(columns={'trip_duration':'trip_duration_wh'},inplace=True)

In [8]:
trip_duration_m = sort_groupby(trip_duration_m)
trip_duration_w = sort_groupby(trip_duration_w)
trip_duration_h = sort_groupby(trip_duration_h)
trip_duration_wh = sort_groupby(trip_duration_wh)

In [9]:
train_data = pd.merge(train_data, trip_duration_m, how='left', on = ['pickup_month'])
train_data = pd.merge(train_data, trip_duration_w, how='left', on = ['pickup_weekday'])
train_data = pd.merge(train_data, trip_duration_h, how='left', on = ['pickup_hour'])
train_data = pd.merge(train_data, trip_duration_wh, how='left', on = ['pickup_weekday','pickup_hour'])

test_data = pd.merge(test_data, trip_duration_m, how='left', on = ['pickup_month'])
test_data = pd.merge(test_data, trip_duration_w, how='left', on = ['pickup_weekday'])
test_data = pd.merge(test_data, trip_duration_h, how='left', on = ['pickup_hour'])
test_data = pd.merge(test_data, trip_duration_wh, how='left', on = ['pickup_weekday','pickup_hour'])

In [10]:
#地理特征
#经纬度离散化（合理？）
n=10
train_data['pickup_longitude_qcut'] = pd.qcut(train_data['pickup_longitude'],n,labels=range(n))
train_data['dropoff_longitude_qcut'] = pd.qcut(train_data['dropoff_longitude'],n,labels=range(n))
train_data['dropoff_latitude_qcut'] = pd.qcut(train_data['dropoff_latitude'],n,labels=range(n))
train_data['pickup_latitude_qcut'] = pd.qcut(train_data['pickup_latitude'],n,labels=range(n))
train_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,trip_duration_w,trip_duration_w_sort,trip_duration_h,trip_duration_h_sort,trip_duration_wh,trip_duration_wh_sort,pickup_longitude_qcut,dropoff_longitude_qcut,dropoff_latitude_qcut,pickup_latitude_qcut
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,0,...,897.947839,7,1030.581619,5,970.433177,64,4,7,6,7
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,0,...,901.639395,6,936.657302,10,913.323961,100,5,1,2,2
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,0,...,983.463124,3,966.430282,9,1055.057076,31,5,0,0,6
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,0,...,975.450494,4,894.356679,17,859.145184,118,0,0,0,0
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,0,...,948.051175,5,1032.246359,4,936.744342,82,6,6,8,9


In [11]:
#地理特征
#经纬度离散化（合理？）
def test_cut(x,a):
    if x<a[1]:
        return 0.0
    elif x<a[2]:
        return 1.0
    elif x<a[3]:
        return 2.0
    elif x<a[4]:
        return 3.0
    elif x<a[5]:
        return 4.0
    elif x<a[6]:
        return 5.0
    elif x<a[7]:
        return 6.0
    elif x<a[8]:
        return 7.0
    elif x<a[9]:
        return 8.0
    elif x< a[10]:
        return 9.0

In [12]:
#地理特征
#经纬度离散化（合理？）
inteval_pickup_longitude = [-121.93334198, -74.0026626587, -73.9942855835, -73.989982605, -73.985786438, -73.9817428589, -73.9772491455, -73.9713821411, -73.9623184204, -73.9518737793, -61.3355293274]
inteval_pickup_latitude= [34.3596954346, 40.7201576233, 40.7322616577, 40.7413101196, 40.7484779358, 40.7541007996, 40.7596817017, 40.764755249, 40.772064209, 40.7804756165, 51.8810844421]
inteval_dropoff_longitude = [-121.933303833, -74.0028610229, -73.9939575195, -73.9890441895, -73.9840927124, -73.9797515869, -73.9750366211, -73.9681091309, -73.9580688477, -73.9472198486, -61.3355293274] 
inteval_dropoff_latitude = [32.1811408997, 40.7153701782, 40.7299880981, 40.7407875061, 40.7485733032, 40.754524231, 40.7601890564, 40.7656936646, 40.7741775513, 40.7848815918, 43.9210281372] 
test_data['pickup_longitude_qcut'] = test_data['pickup_longitude'].map(lambda x:test_cut(x,inteval_pickup_longitude))
test_data['dropoff_longitude_qcut'] = test_data['dropoff_longitude'].map(lambda x:test_cut(x,inteval_dropoff_longitude))
test_data['dropoff_latitude_qcut'] = test_data['dropoff_latitude'].map(lambda x:test_cut(x,inteval_dropoff_latitude))
test_data['pickup_latitude_qcut'] = test_data['pickup_latitude'].map(lambda x:test_cut(x,inteval_pickup_latitude))

In [13]:
#地理特征
#距离特征
def Lon_Lat_distance(lon1,lat1,lon2,lat2):
    deltaLon = np.abs(lon1 - lon2)
    deltaLat = np.abs(lat1 - lat2)
    EARTH_RADIUS = 6371
    disLon,disLat = map(lambda x:np.pi*x*EARTH_RADIUS/180,(deltaLon,deltaLat))
    return disLon, disLat

def calcu_distance(lon1,lat1,lon2,lat2):
    disLon,disLat = Lon_Lat_distance(lon1,lat1,lon2,lat2)
    mht_distance = disLon + disLat
    line_distance = np.sqrt(disLon**2 + disLat**2)
    return mht_distance, line_distance

In [14]:
#地理特征
# 两种距离特征（+2）
train_data['mht_distance'],train_data['line_distance'] = calcu_distance(train_data['pickup_longitude'],train_data['pickup_latitude'],
                                                                  train_data['dropoff_longitude'],train_data['dropoff_latitude'])
test_data['mht_distance'],test_data['line_distance'] = calcu_distance(test_data['pickup_longitude'],test_data['pickup_latitude'],
                                                                  test_data['dropoff_longitude'],test_data['dropoff_latitude'])

In [15]:
train_data.columns.values

array(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'pickup_month', 'pickup_weekday', 'pickup_hour',
       'pickup_number', 'pickup_month_number', 'pickup_month_number_sort',
       'pickup_weekday_number', 'pickup_weekday_number_sort',
       'pickup_hour_number', 'pickup_hour_number_sort',
       'pickup_weekhour_number', 'pickup_weekhour_number_sort',
       'trip_duration_m', 'trip_duration_m_sort', 'trip_duration_w',
       'trip_duration_w_sort', 'trip_duration_h', 'trip_duration_h_sort',
       'trip_duration_wh', 'trip_duration_wh_sort',
       'pickup_longitude_qcut', 'dropoff_longitude_qcut',
       'dropoff_latitude_qcut', 'pickup_latitude_qcut', 'mht_distance',
       'line_distance'], dtype=object)

In [16]:
#地理特征
#不同区域的平均用车时长及排序
region_trip_duration =train_data.groupby(['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['trip_duration'].mean()
region_trip_duration.rename(columns={'trip_duration':'region_trip_duration'},inplace=True)
region_trip_duration_m = train_data.groupby(['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['trip_duration'].mean()
region_trip_duration_m.rename(columns={'trip_duration':'region_trip_duration_m'},inplace=True)
region_trip_duration_w = train_data.groupby(['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['trip_duration'].mean()
region_trip_duration_w.rename(columns={'trip_duration':'region_trip_duration_w'},inplace=True)
region_trip_duration_h = train_data.groupby(['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['trip_duration'].mean()
region_trip_duration_h.rename(columns={'trip_duration':'region_trip_duration_h'},inplace=True)
region_trip_duration_wh = train_data.groupby(['pickup_weekday','pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['trip_duration'].mean()
region_trip_duration_wh.rename(columns={'trip_duration':'region_trip_duration_wh'},inplace=True)


region_trip_duration = sort_groupby(region_trip_duration)
region_trip_duration_m = sort_groupby(region_trip_duration_m)
region_trip_duration_w = sort_groupby(region_trip_duration_w)
region_trip_duration_h = sort_groupby(region_trip_duration_h)
region_trip_duration_wh = sort_groupby(region_trip_duration_wh)






In [17]:
train_data = pd.merge(train_data, region_trip_duration, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, region_trip_duration_m, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, region_trip_duration_w, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, region_trip_duration_h, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, region_trip_duration_wh, how='left', on = ['pickup_weekday','pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])


KeyError: 'pickup_weekday'

In [ ]:
test_data = pd.merge(test_data, region_trip_duration, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, region_trip_duration_m, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, region_trip_duration_w, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, region_trip_duration_h, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, region_trip_duration_wh, how='left', on = ['pickup_weekday','pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])

In [ ]:
# 地理&时间特征
# 叫车量
# pickup_region_number
# pickup_region_number_m
# pickup_region_number_w
# pickup_region_number_h
# pickup_region_number_wh

train_data['pickup_number'] = 1 # 后续删除
pickup_region_number = train_data.groupby(['pickup_longitude_qcut','pickup_latitude_qcut',
                                        'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['pickup_number'].sum()
pickup_region_number.rename(columns={'pickup_number':'pickup_region_number'},inplace=True)
pickup_region_number_m = train_data.groupby(['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                        'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['pickup_number'].sum()
pickup_region_number_m.rename(columns={'pickup_number':'pickup_region_number_m'},inplace=True)
pickup_region_number_w = train_data.groupby(['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                          'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['pickup_number'].sum()
pickup_region_number_w.rename(columns={'pickup_number':'pickup_region_number_w'},inplace=True)
pickup_region_number_h = train_data.groupby(['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                          'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['pickup_number'].sum()
pickup_region_number_h.rename(columns={'pickup_number':'pickup_region_number_h'},inplace=True)
pickup_region_number_wh = train_data.groupby(['pickup_weekday','pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                           'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['pickup_number'].sum()
pickup_region_number_wh.rename(columns={'pickup_number':'pickup_region_number_wh'},inplace=True)

pickup_region_number = sort_groupby(pickup_region_number)
pickup_region_number_m = sort_groupby(pickup_region_number_m)
pickup_region_number_w = sort_groupby(pickup_region_number_w)
pickup_region_number_h = sort_groupby(pickup_region_number_h)
pickup_region_number_wh = sort_groupby(pickup_region_number_wh)

In [ ]:
train_data = pd.merge(train_data, pickup_region_number, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, pickup_region_number_m, how='left', on = ['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, pickup_region_number_w, how='left', on = ['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, pickup_region_number_h, how='left', on = ['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, pickup_region_number_wh, how='left', on = ['pickup_weekday','pickup_hour','pickup_longitude_qcut',
                                                                   'pickup_latitude_qcut','dropoff_longitude_qcut','dropoff_latitude_qcut'])

test_data = pd.merge(test_data, pickup_region_number, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, pickup_region_number_m, how='left', on = ['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, pickup_region_number_w, how='left', on = ['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, pickup_region_number_h, how='left', on = ['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, pickup_region_number_wh, how='left', on = ['pickup_weekday','pickup_hour','pickup_longitude_qcut',
                                                                   'pickup_latitude_qcut','dropoff_longitude_qcut','dropoff_latitude_qcut'])

In [ ]:
#地理特征
#不同区域的平均距离及排序
region_mht_distance = train_data.groupby(['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['mht_distance'].mean()
region_mht_distance.rename(columns={'mht_distance':'region_mht_distance'},inplace=True)
region_line_distance = train_data.groupby(['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['line_distance'].mean()
region_line_distance.rename(columns={'line_distance':'region_line_distance'},inplace=True)

region_mht_distance = region_mht_distance.sort_values(by='region_mht_distance',ascending=False)
region_mht_distance['region_mht_distance_sort'] = range(1,len(region_mht_distance)+1)
region_line_distance = region_line_distance.sort_values(by='region_line_distance',ascending=False)
region_line_distance['region_line_distance_sort'] = range(1,len(region_line_distance)+1)

train_data = pd.merge(train_data, region_mht_distance, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, region_mht_distance, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, region_line_distance, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, region_line_distance, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])

In [ ]:
#地理特征
#经纬度差、运行方向
train_data['longitude_delta'] = train_data['dropoff_longitude'] - train_data['pickup_longitude']
train_data['latitude_delta'] = train_data['dropoff_latitude'] - train_data['pickup_latitude']
train_data['longitude_dct'] = (train_data.longitude_delta > 0)*1
train_data['latitude_dct'] = (train_data.latitude_delta > 0)*1

test_data['longitude_delta'] = test_data['dropoff_longitude'] - test_data['pickup_longitude']
test_data['latitude_delta'] = test_data['dropoff_latitude'] - test_data['pickup_latitude']
test_data['longitude_dct'] = (test_data.longitude_delta > 0)*1
test_data['latitude_dct'] = (test_data.latitude_delta > 0)*1

In [ ]:
#速度特征
train_data['mht_speed'] = 1000 * train_data['mht_distance'] / train_data['trip_duration']
train_data['line_speed'] = 1000 * train_data['line_distance'] / train_data['trip_duration']

In [4]:
#速度特征
#基于地理的速度、基于地理和时间的速度
#直线速度、曼哈特速度
mht_region_speed = train_data.groupby(['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['mht_speed'].mean()
mht_region_speed.rename(columns={'mht_speed':'mht_region_speed'},inplace=True)
mht_region_speed_m = train_data.groupby(['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['mht_speed'].mean()
mht_region_speed_m.rename(columns={'mht_speed':'mht_region_speed_m'},inplace=True)
mht_region_speed_w = train_data.groupby(['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['mht_speed'].mean()
mht_region_speed_w.rename(columns={'mht_speed':'mht_region_speed_w'},inplace=True)
mht_region_speed_h = train_data.groupby(['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['mht_speed'].mean()
mht_region_speed_h.rename(columns={'mht_speed':'mht_region_speed_h'},inplace=True)
mht_region_speed_wh = train_data.groupby(['pickup_weekday','pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['mht_speed'].mean()
mht_region_speed_wh.rename(columns={'mht_speed':'mht_region_speed_wh'},inplace=True)

line_region_speed = train_data.groupby(['pickup_longitude_qcut','pickup_latitude_qcut',
                                                      'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['line_speed'].mean()
line_region_speed.rename(columns={'line_speed':'line_region_speed'},inplace=True)
line_region_speed_m = train_data.groupby(['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['line_speed'].mean()
line_region_speed_m.rename(columns={'line_speed':'line_region_speed_m'},inplace=True)
line_region_speed_w = train_data.groupby(['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['line_speed'].mean()
line_region_speed_w.rename(columns={'line_speed':'line_region_speed_w'},inplace=True)
line_region_speed_h = train_data.groupby(['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['line_speed'].mean()
line_region_speed_h.rename(columns={'line_speed':'line_region_speed_h'},inplace=True)
line_region_speed_wh = train_data.groupby(['pickup_weekday','pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['line_speed'].mean()
line_region_speed_wh.rename(columns={'line_speed':'line_region_speed_wh'},inplace=True)

#排序
mht_region_speed = sort_groupby(mht_region_speed)
mht_region_speed_m = sort_groupby(mht_region_speed_m)
mht_region_speed_w = sort_groupby(mht_region_speed_w)
mht_region_speed_h = sort_groupby(mht_region_speed_h)
mht_region_speed_wh = sort_groupby(mht_region_speed_wh)
line_region_speed = sort_groupby(line_region_speed)
line_region_speed_m = sort_groupby(line_region_speed_m)
line_region_speed_w = sort_groupby(line_region_speed_w)
line_region_speed_h = sort_groupby(line_region_speed_h)
line_region_speed_wh = sort_groupby(line_region_speed_wh)

NameError: name 'train_data' is not defined

In [ ]:
train_data = pd.merge(train_data, mht_region_speed, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, mht_region_speed_m, how='left', on = ['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, mht_region_speed_w, how='left', on = ['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, mht_region_speed_h, how='left', on = ['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, mht_region_speed_wh, how='left', on = ['pickup_weekday','pickup_hour','pickup_longitude_qcut',
                                                                   'pickup_latitude_qcut','dropoff_longitude_qcut','dropoff_latitude_qcut'])

train_data = pd.merge(train_data, line_region_speed, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, line_region_speed_m, how='left', on = ['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, line_region_speed_w, how='left', on = ['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, line_region_speed_h, how='left', on = ['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, line_region_speed_wh, how='left', on = ['pickup_weekday','pickup_hour','pickup_longitude_qcut',
                                                                   'pickup_latitude_qcut','dropoff_longitude_qcut','dropoff_latitude_qcut'])

In [ ]:
test_data = pd.merge(test_data, mht_region_speed, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, mht_region_speed_m, how='left', on = ['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, mht_region_speed_w, how='left', on = ['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, mht_region_speed_h, how='left', on = ['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, mht_region_speed_wh, how='left', on = ['pickup_weekday','pickup_hour','pickup_longitude_qcut',
                                                                   'pickup_latitude_qcut','dropoff_longitude_qcut','dropoff_latitude_qcut'])

test_data = pd.merge(test_data, line_region_speed, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, line_region_speed_m, how='left', on = ['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, line_region_speed_w, how='left', on = ['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, line_region_speed_h, how='left', on = ['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
test_data = pd.merge(test_data, line_region_speed_wh, how='left', on = ['pickup_weekday','pickup_hour','pickup_longitude_qcut',
                                                                   'pickup_latitude_qcut','dropoff_longitude_qcut','dropoff_latitude_qcut'])

In [ ]:
#速度特征
#基于时间的速度
mht_speed_m = train_data.groupby(['pickup_month'],as_index=False)['mht_speed'].mean()
mht_speed_m.rename(columns={'mht_speed':'mht_speed_m'},inplace=True)
mht_speed_w = train_data.groupby(['pickup_weekday'],as_index=False)['mht_speed'].mean()
mht_speed_w.rename(columns={'mht_speed':'mht_speed_w'},inplace=True)
mht_speed_h = train_data.groupby(['pickup_hour'],as_index=False)['mht_speed'].mean()
mht_speed_h.rename(columns={'mht_speed':'mht_speed_h'},inplace=True)
mht_speed_wh = train_data.groupby(['pickup_weekday','pickup_hour'],as_index=False)['mht_speed'].mean()
mht_speed_wh.rename(columns={'mht_speed':'mht_speed_wh'},inplace=True)

line_speed_m = train_data.groupby(['pickup_month'],as_index=False)['line_speed'].mean()
line_speed_m.rename(columns={'line_speed':'line_speed_m'},inplace=True)
line_speed_w = train_data.groupby(['pickup_weekday'],as_index=False)['line_speed'].mean()
line_speed_w.rename(columns={'line_speed':'line_speed_w'},inplace=True)
line_speed_h = train_data.groupby(['pickup_hour'],as_index=False)['line_speed'].mean()
line_speed_h.rename(columns={'line_speed':'line_speed_h'},inplace=True)
line_speed_wh = train_data.groupby(['pickup_weekday','pickup_hour'],as_index=False)['line_speed'].mean()
line_speed_wh.rename(columns={'line_speed':'line_speed_wh'},inplace=True)

#排序
mht_speed_m = sort_groupby(mht_speed_m)
mht_speed_w = sort_groupby(mht_speed_w)
mht_speed_h = sort_groupby(mht_speed_h)
mht_speed_wh = sort_groupby(mht_speed_wh)
line_speed_m = sort_groupby(line_speed_m)
line_speed_w = sort_groupby(line_speed_w)
line_speed_h = sort_groupby(line_speed_h)
line_speed_wh = sort_groupby(line_speed_wh)

In [ ]:
train_data = pd.merge(train_data, mht_speed_m, how='left', on = ['pickup_month'])
train_data = pd.merge(train_data, mht_speed_w, how='left', on = ['pickup_weekday'])
train_data = pd.merge(train_data, mht_speed_h, how='left', on = ['pickup_hour'])
train_data = pd.merge(train_data, mht_speed_wh, how='left', on = ['pickup_weekday','pickup_hour'])

train_data = pd.merge(train_data, line_speed_m, how='left', on = ['pickup_month'])
train_data = pd.merge(train_data, line_speed_w, how='left', on = ['pickup_weekday'])
train_data = pd.merge(train_data, line_speed_h, how='left', on = ['pickup_hour'])
train_data = pd.merge(train_data, line_speed_wh, how='left', on = ['pickup_weekday','pickup_hour'])

In [ ]:
test_data = pd.merge(test_data, mht_speed_m, how='left', on = ['pickup_month'])
test_data = pd.merge(test_data, mht_speed_w, how='left', on = ['pickup_weekday'])
test_data = pd.merge(test_data, mht_speed_h, how='left', on = ['pickup_hour'])
test_data = pd.merge(test_data, mht_speed_wh, how='left', on = ['pickup_weekday','pickup_hour'])

test_data = pd.merge(test_data, line_speed_m, how='left', on = ['pickup_month'])
test_data = pd.merge(test_data, line_speed_w, how='left', on = ['pickup_weekday'])
test_data = pd.merge(test_data, line_speed_h, how='left', on = ['pickup_hour'])
test_data = pd.merge(test_data, line_speed_wh, how='left', on = ['pickup_weekday','pickup_hour'])

In [ ]:
# get_dummies 月、日、时、区域
train_data = pd.get_dummies(train_data,prefix=['pickup_month','pickup_weekday','pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                          'dropoff_longitude_qcut','dropoff_latitude_qcut'],
                          columns=['pickup_month','pickup_weekday','pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                   'dropoff_longitude_qcut','dropoff_latitude_qcut'])

test_data = pd.get_dummies(test_data,prefix=['pickup_month','pickup_weekday','pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                          'dropoff_longitude_qcut','dropoff_latitude_qcut'],
                          columns=['pickup_month','pickup_weekday','pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                   'dropoff_longitude_qcut','dropoff_latitude_qcut'])

In [ ]:
del train_data['dropoff_datetime']
del train_data['mht_speed']
del train_data['line_speed']
del train_data['pickup_number']
del train_data['id']
del train_data['pickup_datetime']
del train_data['pickup_longitude']
del train_data['pickup_latitude']
del train_data['dropoff_longitude']
del train_data['dropoff_latitude']
del test_data['id']
del test_data['pickup_datetime']
del test_data['pickup_longitude']
del test_data['pickup_latitude']
del test_data['dropoff_longitude']
del test_data['dropoff_latitude']

In [11]:
train_data.to_csv('train_data_lgc_1.csv',index=False)

In [2]:
train_data = pd.read_csv('train_data_lgc_1.csv')